14-数组的索引参考
====
NumPy Reference ---> Array objects ---> Indexing

可以使用标准的 Python `x [obj]` 语法对 ndarrays 进行检索，其中 x 是数组，obj 是选择。 有三种检索方式可用：
- 字段访问，
- 基本切片，
- 高级索引。

引发哪一个检索方式取决于 `obj`。

**注意：**
在Python中， x[(exp1, exp2, ..., expN)] 等同于 x[exp1, exp2, ..., expN]; 后者只是对前者的语法糖。

# 基本切片和索引

基本切片将Python的切片基本概念扩展到 N 维。当 obj 是：
- 切片对象（由圆括号内的`start:stop:step`符号构造），
- 整数，
- 或切片对象和整数构成的元组时，

会引发“基本切片”式检索。Ellipsis 省略号和 newaxis 对象也可以穿插其中。

使用 N 个整数进行检索的最简单情况是返回表示相应项的数组标量。与在Python中一样，所有索引都是从零开始的：对于第i个索引n_i，有效范围是 `0 <= n_i < d_i`，其中 `d_i` 是数组 shape 的第i个元素。负索引值被解释为从数组的末尾开始计数（即，如果`n_i <0`，则表示`n_i + d_i`）。

通过基本切片生成的所有数组始终是原始数组的视图。

**注：** 

在批处理中>和<为重定向符号，这就意味着不能用 > 来表示大于，< 表示小于，也就意味着不能用 >=、<=、<> 来表示大于等于、小于等于、不等于，还好，在批处理中用了其他的操作符代替它们：

- EQU, 等于
- NEQ, 不等于
- LSS, 小于
- LEQ, 小于或等于
- GTR, 大于
- GEQ, 大于或等于

序列切片的标准规则适用于基于每维的基本切片（包括使用step索引）。要记住的一些有用的概念包括：
- 基本切片语法是 `i：j：k` 其中i是起始索引，j是停止索引，k是步长（`k <> 0`）。 这样就选择了m个元素（在相应的维度中），索引值依次是 `i, i+k, …, i+(m-1)k `，其中 `m = q+r (r<>0)`，q 和 r 分别是`j-i / k`得到的整数商和余数。
`j-i = q*k+r`, 所以 `i+(m-1)k < j`.

In [1]:
# Example
import numpy as np
x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
x[1:7:2]

array([1, 3, 5])

- 负i和j被解释为 `n+i` 和 `n+j`，其中 n 是相应维度中的元素数量。负k使得 step 指向更小的索引值。即反向索引。

In [2]:
>>> x[-2:10]

array([8, 9])

In [3]:
>>> x[-3:3:-1]

array([7, 6, 5, 4])

- 假设 n 是要切片的维度中的元素数。如果没有给出 i，则 k>0 时默认为 0，k < 0 时为 n-1。 如果没有给出 j，则对于k> 0，默认为 j=n，对于 k<0，默认为 `j = -n-1`。如果没有给出k，则默认为 k=1。注意 `::` 与 `:` 相同，并且表示在该轴上选择所有索引。

In [4]:
x[:5]

array([0, 1, 2, 3, 4])

- 如果选择元组中的对象数量小于N，则 `:`被假定用于后续所有维度（即后续维度选择全部子元素）：

**注**：对于多维数组（D>2）,`x[i:j:k]`这种基本索引方法，首先在0维度上，根据规则选择行，剩余维度上使用`[:]`这样的索引，即后续维度上索引全部元素。

In [5]:
# Example
>>> x = np.array([[[1],[2],[3]], [[4],[5],[6]]])
>>> x.shape

(2, 3, 1)

In [6]:
x

array([[[1],
        [2],
        [3]],

       [[4],
        [5],
        [6]]])

In [7]:
>>> x[1:2]

array([[[4],
        [5],
        [6]]])

In [8]:
y=np.arange(48)
y = y.reshape(2,3,2,4)
y

array([[[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7]],

        [[ 8,  9, 10, 11],
         [12, 13, 14, 15]],

        [[16, 17, 18, 19],
         [20, 21, 22, 23]]],


       [[[24, 25, 26, 27],
         [28, 29, 30, 31]],

        [[32, 33, 34, 35],
         [36, 37, 38, 39]],

        [[40, 41, 42, 43],
         [44, 45, 46, 47]]]])

In [9]:
y[1:3]

array([[[[24, 25, 26, 27],
         [28, 29, 30, 31]],

        [[32, 33, 34, 35],
         [36, 37, 38, 39]],

        [[40, 41, 42, 43],
         [44, 45, 46, 47]]]])

- Ellipsis省略号扩展的 `：`对象的数量必需生成与`x.ndim`长度相同的选择元组。可能只存在一个省略号。

In [10]:
 x[...,0]

array([[1, 2, 3],
       [4, 5, 6]])

In [11]:
x.ndim

3

- 选择元组中的每个newaxis对象用于将选择结果的维度扩展一个单位长度维度。添加的维度是 newaxis 对象在选择元组中的位置。

In [13]:
 x[:,np.newaxis,:,:].shape

(2, 1, 3, 1)

- 整数 i 返回与 `i：i + 1`相同的值，除了返回对象的维数减少 1。特别是，具有第 p 个元素的整数（和所有其他`:`条目)的选择元组返回对应的子数组，其维数为 `N - 1`。如果 N = 1，则返回的对象是数组标量。

- 如果选择元组具有所有 `：`条目，除了作为切片对象`i：j：k`的第 p 个条目之外，则返回的数组有 N 个维度，连接通过元素为`i, i+k, …, i + (m - 1) k < j`的整数索引返回的子数组构成。

- 在切片元组中使用多个非`：`条目进行基本切片，就像重复应用单个非`：`条目的切片一样，非`:`条目被连续使用，（所有其他非`：`条目替换为`:`)。 因此，`x [ind1，...，ind2，：]`在基本切片下的行为类似于`x [ind1] [...，ind2，：]`。

**警告**
对于高级索引，上述情况并非如此。

- 可以使用切片来设置数组中的值，但是（与列表不同）永远不会增长数组。要在`x [obj] = value`中设置的值的大小必须（可广播）为与`x [obj]`相同的形状。

**注意**
切片元组总是可以构造为obj并在`x [obj]`表示法中使用。可以在构造中使用切片对象代替`[start：stop：step]`表示法。 例如，`x [1：10：5，:: - 1]`也可以实现为`obj =（slice（1,10,5），slice（None，None，-1））; x [obj]`。 这对于构造适用于任意维数组的通用代码非常有用。

**numpy.newaxis**

newaxis对象可用于所有切片操作，以创建长度为1的轴。 newaxis是'None'的别名，'None'可以代替它使用得到相同的结果。

# 高级检索
下面两种情形触发高级检索：
- 1.选择对象 obj 是非元组序列对象时，（不是元组对象，是其它序列对象，如列表。）
- 2.选择对象 obj 是具有至少一个序列对象或ndarray（数据类型为integer或bool）的元组时。（元组，元组中至少有一个序列对象或ndarray。）

**注意：**判断是否为元组和序列，要对应检索位看。如果选择对象obj是 `x[obj]`的`[ ]`中的内容。

高级检索有两种类型：整数 和 布尔值。

高级索引始终返回数据的副本（与返回视图的基本切片形成对比）。


**警告**
高级索引的定义意味着`x [（1,2,3），]` <span class="burk">（`obj=(1,2,3),`，元组中包含一个序列，触发高级检索。）</span> 与 `x [（1,2,3）]`（<span class="burk">`obj=(1,2,3)`，是元组，但每个元素都是单个值，没有序列或ndarray</span>） 根本不同。后者相当于`x [1,2,3]`，它将触发基本选择，而前者将触发高级索引。 一定要明白为什么会这样。

还要认识到`x [[1,2,3]]`（`obj=[1,2,3]`是一个列表，情形1，非元组序列。）将触发高级索引，而由于上面提到的弃用的数值兼容性，x [[1,2，slice（None）]]将触发基本切片。

**判断是否触发高级检索的步骤：**

obj是否是元组？
1. 不是元组，再判断：
> 是否是序列对象?
    >>是序列对象，触发高级检索；    
    >>不是序列对象，触发基本检索。
2. 是元组，再判断：
> 是否包含至少一个序列元素或 ndarray 元素？
    >> 包含，触发高级检索；    
    >> 不包含，触发基本检索。

## 整数数组检索
整数数组索引允许基于数组的 N-D索引选择数组的任意项，每个整数数组表示该维度的多个索引。

### 纯整数数组检索

当索引值包含与被检索数组的维度数一样的整数数组时，检索是直接了当的，但与切片不同。

高级索引始终作为一个广播和迭代：

高级索引始终作为一个广播和迭代：

`result[i_1, ..., i_M] == x[ind_1[i_1, ..., i_M], ind_2[i_1, ..., i_M],
                           ..., ind_N[i_1, ..., i_M]]`

**注意**，结果形状与（广播）索引数组形状ind_1，...，ind_N相同。

**例**
从每一行开始，应选择一个特定元素。 行索引只是[0,1,2]，列索引指定要为相应行选择的元素，此处为[0,1,0]。 将两者结合使用可以使用高级索引解决任务：

In [15]:
>>> x = np.array([[1, 2], [3, 4], [5, 6]])
x

array([[1, 2],
       [3, 4],
       [5, 6]])

In [16]:
>>> x[[0, 1, 2], [0, 1, 0]]

array([1, 4, 5])

为了实现类似于上面的基本切片的行为，可以使用广播。 函数`ix_`可以帮助这个广播。通过示例可以最好地理解这一点。

**例**
从 4x3 数组中选择角元素，应使用高级检索。因此，需要选择列为[0,2]之一且行为[0,3]之一的所有元素。要使用高级检索，需要显式地选择所有元素。使用前面解释的方法可以这样写：

In [21]:
>>> x = np.array([[ 0,  1,  2],
...            [ 3,  4,  5],
...            [ 6,  7,  8],
...            [ 9, 10, 11]])
x

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [25]:
>>> rows = np.array([[0, 0],
                     [3, 3]], dtype=np.intp)

In [26]:
>>> columns = np.array([[0, 2],
                        [0, 2]], dtype=np.intp)

In [27]:
>>> x[rows, columns]

array([[ 0,  2],
       [ 9, 11]])

但是，由于上面的索引数组只是重复自身，因此可以使用广播（比较诸如`rows [：，np.newaxis] + columns`之类的操作）来简化：

In [32]:
>>> rows = np.array([0, 3], dtype=np.intp)

In [33]:
>>> columns = np.array([0, 2], dtype=np.intp)

In [34]:
>>> rows[:, np.newaxis]

array([[0],
       [3]], dtype=int64)

In [29]:
>>> x[rows[:, np.newaxis], columns]

array([[ 0,  2],
       [ 9, 11]])

这个广播也可以使用`ix_`函数来实现：

In [35]:
x[np.ix_(rows, columns)]

array([[ 0,  2],
       [ 9, 11]])

请注意，如果没有np.ix_调用，则只会选择对角线元素，如上例所示。 对于使用多个高级索引进行索引，这个差异是最重要的。